In [1]:
import io
import subprocess
import os
import glob
import sys
import pwd

import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
from sklearn.decomposition import PCA
import pickle
import re
import math
import plotnine as pn
import seaborn as sns
import glob
import subprocess
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from numpy import log
from collections import defaultdict,Counter
import re
import statsmodels
import warnings
import statsmodels.stats.multitest as smm
import json

from pathlib import Path
#from midas import m
#from datetime import datetime, date
#import altair as alt

# gh specific
sys.path.append('/ghsfa/projects/pharma/shared_scripts/')
import bibp.functions as functions
import hgvslib.pHGVS as pHGVS  ## do a pull on the hgvslib git repo

# !pip install statannotations==0.4.0
# #!pip uninstall statannotations --yes
# from statannotations.Annotator import Annotator

warnings.filterwarnings("ignore")
pd.options.display.max_rows = None
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None
work_dir="/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/CRC_methylation_caller_result_check_01202025"

## Data preocessing

In [2]:
# Readin the Infinity batch
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20241114_AstraZeneca_SOW202_Batch1_80Samples_InfinityReport.csv", header = 1) 

# Jill's batches
# df_ori = pd.read_csv("/ghsfa/projects/pharma/customers/Pfizer/Infinity/PFE_SOW51core_Infinity/PFE_SOW51core_Infinity_03/data/reports/20250121_Pfizer_SOW51core_Batch3_18Samples_InfinityReport.csv", header = 1) 
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20250117_Pfizer_SOW46Amendment1TP1_Batch9_1Samples_InfinityReport.csv", header = 1) 
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20250117_Pfizer_SOW46Amendment1TP1_Batch10_3Samples_InfinityReport.csv", header = 1) 
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20250120_Pfizer_SOW46Amendment1TP3_Batch9_20Samples_G360Report.csv", header = 1) 
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20250120_Pfizer_SOW46Amendment1TP3_Batch10_8Samples_G360Report.csv", header = 1) 
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20250117_Pfizer_SOW46Amendment1TP5_Batch6_11Samples_InfinityReport.csv", header = 1) 
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20250120_Pfizer_SOW46Amendment1TP5_Batch7_33Samples_InfinityReport.csv", header = 1) 


# Mingyang's batches
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/data/infinity_report_warehouse/20241220_GuardantHealthIST_IST01_Batch1_405Samples_InfinityReport.csv", header = 1) 

# Haiyang's batch
df_ori = pd.read_csv("/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/XTX_TFv2_update_Batch1to40_01162025/df_xtx_Sirius161_03062025.csv", header = 0) 


print("Total sample size of the Infinity batch is " + str(df_ori.shape[0]) + "\n") 
print("Total unique sample size of the Infinity batch is " + str(df_ori.GHSampleID.nunique()) + "\n")
print("Total unique patient sample size of the Infinity batch is " + str(df_ori.Patient_ID.nunique()) + "\n")

df_ori.head()

Total sample size of the Infinity batch is 7

Total unique sample size of the Infinity batch is 7

Total unique patient sample size of the Infinity batch is 5



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date
0,XTX101-0102-001,NaN,A1277545,A127754501,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,19.86,NaN,NaN,1/24/25,1/23/25,2/11/25
1,XTX101-0102-001,NaN,A1285830,A128583001,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,27.23,NaN,NaN,1/31/25,1/30/25,2/19/25
2,XTX101-0102-001,NaN,A1243649,A124364901,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,21.39,NaN,NaN,12/18/24,12/17/24,1/2/25
3,XTX101-0102-001,NaN,A1247604,A124760401,146-005,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,26.64,NaN,NaN,12/21/24,12/19/24,1/2/25
4,XTX101-0102-001,NaN,A1276510,A127651001,140-008,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,33.94,NaN,NaN,1/23/25,1/22/25,2/11/25


In [3]:
# Remove duplication
df_dedup = df_ori.drop_duplicates(subset = "GHSampleID")
df_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date
0,XTX101-0102-001,NaN,A1277545,A127754501,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,19.86,NaN,NaN,1/24/25,1/23/25,2/11/25
1,XTX101-0102-001,NaN,A1285830,A128583001,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,27.23,NaN,NaN,1/31/25,1/30/25,2/19/25
2,XTX101-0102-001,NaN,A1243649,A124364901,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,21.39,NaN,NaN,12/18/24,12/17/24,1/2/25
3,XTX101-0102-001,NaN,A1247604,A124760401,146-005,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,26.64,NaN,NaN,12/21/24,12/19/24,1/2/25
4,XTX101-0102-001,NaN,A1276510,A127651001,140-008,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,33.94,NaN,NaN,1/23/25,1/22/25,2/11/25


In [4]:
# Pull "runid" from ghdb
df_1 = df_dedup.copy()

# Define the IDs you want to query for
ids = tuple(df_1['GHSampleID'].unique())

# Generate the SQL query string
query = "SELECT * FROM gh_sample WHERE run_sample_id IN {}".format(ids)

# Get the dataframe with "runid"
sample_df = functions.run_sql_query(query)
sample_df = sample_df.rename(columns={"run_sample_id": "GHSampleID"})

# Found some sample has duplicated runid which might due to re-run. So only the latest FC runs were saved.
# Extract the date from runid and convert to datetime format for accurate comparison
sample_df['run_date'] = pd.to_datetime(sample_df['runid'].str.extract(r'(\d{6})')[0], format='%y%m%d')
df_1 = df_1.merge(sample_df[['GHSampleID', 'runid', 'run_date']], on='GHSampleID', how="left")
df_1 = df_1.sort_values(by=['GHSampleID', 'run_date'], ascending=[True, False]).drop_duplicates(subset='GHSampleID', keep='first')
df_1 = df_1.drop(columns='run_date')

print("Total sample size of the Infinity batch after merging runid is " + str(df_1.shape[0])) 
df_1.head()


Total sample size of the Infinity batch after merging runid is 7


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid
2,XTX101-0102-001,NaN,A1243649,A124364901,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,21.39,NaN,NaN,12/18/24,12/17/24,1/2/25,241221_A01020_0945_AHWTCMDSXC
3,XTX101-0102-001,NaN,A1247604,A124760401,146-005,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,26.64,NaN,NaN,12/21/24,12/19/24,1/2/25,241227_A02019_0258_BHWTKCDSXC
5,XTX101-0102-001,NaN,A1263902,A126390201,141-006,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,11.10,NaN,NaN,1/11/25,1/9/25,2/4/25,250114_A01021_0837_AH27KNDSXF
4,XTX101-0102-001,NaN,A1276510,A127651001,140-008,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,33.94,NaN,NaN,1/23/25,1/22/25,2/11/25,250125_A01744_0377_AH27JKDSXF
0,XTX101-0102-001,NaN,A1277545,A127754501,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,19.86,NaN,NaN,1/24/25,1/23/25,2/11/25,250126_A01021_0847_BH272FDSXF


In [ ]:
# # Optional step!!!
# # If you only have one sample in the batch, run this cell alternatively instead of from above one

# # Pull "runid" from ghdb
# df_1 = df_dedup.copy()

# # Define the IDs you want to query for
# ids = tuple(df_1['GHSampleID'].unique())

# # If there's only one ID, format it without a comma
# if len(ids) == 1:
#     ids = f"('{ids[0]}')"
# else:
#     ids = str(ids)

# # Generate the SQL query string
# query = f"SELECT * FROM gh_sample WHERE run_sample_id IN {ids}"

# # Get the dataframe with "runid"
# sample_df = functions.run_sql_query(query)
# sample_df = sample_df.rename(columns={"run_sample_id": "GHSampleID"})

# # Found some sample has duplicated runid which might due to re-run. So only the latest FC runs were saved.
# # Extract the date from runid and convert to datetime format for accurate comparison
# sample_df['run_date'] = pd.to_datetime(sample_df['runid'].str.extract(r'(\d{6})')[0], format='%y%m%d')
# df_1 = df_1.merge(sample_df[['GHSampleID', 'runid', 'run_date']], on='GHSampleID', how="left")
# df_1 = df_1.sort_values(by=['GHSampleID', 'run_date'], ascending=[True, False]).drop_duplicates(subset='GHSampleID', keep='first')
# df_1 = df_1.drop(columns='run_date')

# print("Total sample size of the Infinity batch after merging runid is " + str(df_1.shape[0]))
# df_1.head()


Total sample size of the Infinity batch after merging runid is 1


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid
0,C4221015,13431002_Cycle 1 Day 1,A1218829,B00888193,13431002,Cycle 1 Day 1,1,SUCCESS,NaN,SNV,NaN,PIN1,19,9958804.0,3.0,A124S,G>T,c.370G>T,NM_006221.4,51.12,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs138056875,NaN,NaN,NaN,NaN,5935.0,5.85,NaN,0.060326,Detected,0.056315,22.78,High,Not Detected,22.85,3.5,0.0,11/16/2024,06/20/2024,01/17/2025,CRC,Not Applicable,Not Applicable,250106_A00277_1208_BHWTHFDSXC


In [5]:
def add_fc_dir_column(df):
    """
    Adds a 'fc_dir' column to the given dataframe, specifying the FC path based on the 'runid' column.

    Args:
        df (pd.DataFrame): A DataFrame that must contain a 'runid' column. This column is used to
                           match directory paths from predefined locations.

    Returns:
        pd.DataFrame: The original dataframe with an added 'fc_dir' column indicating the directory path
                      corresponding to each 'runid'. If no directory is found, the path will be an empty string.
        pd.DataFrame: A DataFrame containing the rows for which no directory path was found.
    """
    # Define the path
    fc_dir_all = "/ghsfa/ivd/flowcentral"
    fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"
    
    # Initialize a list to store rows where no BIP output was found
    rows_no_found = []
    
    # Initialize the new column in the DataFrame
    df['fc_dir'] = ''
    
    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        runid = row["runid"]
        
        # Find original folder
        pattern_ori = f"{fc_dir_all}/*{runid}*"
        folders = glob.glob(pattern_ori)
        if folders:
            for folder in folders:
                primary_fc_dir = folder
                # Check if there is a 'sirius' folder under the primary fc_dir. This is for Sirius 1.5 folder structure change.
                if os.path.exists(os.path.join(primary_fc_dir, "sirius")):
                    primary_fc_dir = os.path.join(primary_fc_dir, "sirius")
                df.loc[index, 'fc_dir'] = primary_fc_dir
        else:
            # Search in the alternative directory if no folder found in the first directory
            pattern_omni = f"{fc_dir_omni}/*{runid}*"
            folders_omni = glob.glob(pattern_omni)
            if folders_omni:
                df.loc[index, 'fc_dir'] = folders_omni[0]
            else:
                # Append the row to the list if no folder is found in either location
                rows_no_found.append(row)
    
    # Create a DataFrame with rows where folders were not found in any sources
    df_no_fc_dir = pd.DataFrame(rows_no_found)
    
    return df, df_no_fc_dir

# Generate the new dataframe 
df_2, df_no_found = add_fc_dir_column(df_1)

print("Total sample size of the Infinity batch with fc_dir is " + str(df_2.shape[0]) + "\n") 
print("Total sample size of the Infinity batch with NO fc_dir is " + str(df_no_found.shape[0]) + "\n") 

df_2.head()



Total sample size of the Infinity batch with fc_dir is 7

Total sample size of the Infinity batch with NO fc_dir is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid,fc_dir
2,XTX101-0102-001,NaN,A1243649,A124364901,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,21.39,NaN,NaN,12/18/24,12/17/24,1/2/25,241221_A01020_0945_AHWTCMDSXC,/ghsfa/ivd/flowcentral/241221_A01020_0945_AHWTCMDSXC/sirius
3,XTX101-0102-001,NaN,A1247604,A124760401,146-005,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,26.64,NaN,NaN,12/21/24,12/19/24,1/2/25,241227_A02019_0258_BHWTKCDSXC,/ghsfa/ivd/flowcentral/241227_A02019_0258_BHWTKCDSXC/sirius
5,XTX101-0102-001,NaN,A1263902,A126390201,141-006,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,11.10,NaN,NaN,1/11/25,1/9/25,2/4/25,250114_A01021_0837_AH27KNDSXF,/ghsfa/ivd/flowcentral/250114_A01021_0837_AH27KNDSXF/sirius
4,XTX101-0102-001,NaN,A1276510,A127651001,140-008,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,33.94,NaN,NaN,1/23/25,1/22/25,2/11/25,250125_A01744_0377_AH27JKDSXF,/ghsfa/ivd/flowcentral/250125_A01744_0377_AH27JKDSXF/sirius
0,XTX101-0102-001,NaN,A1277545,A127754501,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,19.86,NaN,NaN,1/24/25,1/23/25,2/11/25,250126_A01021_0847_BH272FDSXF,/ghsfa/ivd/flowcentral/250126_A01021_0847_BH272FDSXF.0e716317-4529-4481-89c6-10be60fc1006.20250128124242/sirius


## Generate S3 MB caller outputs using Sirius 1.6.x yaml
###### Git clone "Sirius-1.6.0_Release_Branch" and "devel/mcai/update_crc_model_file" (Sirius-1.6.1) of bip-products for corresponding CRC caller cutoffs and correct CRC region_file for MB CRC caller
###### bip-epigenomics module instead of epi-caller was used for CRC result comparison

In [ ]:
# Instead of using epicaller.methylation.methyl_caller funciton, bip-epigenomics module (branch 2.5) is prefered.

def run_bip_epigenomics_module_wrapper_slurm_Sirius161(sample_meta_df, workdir):
    module_dir = '/home/mcai/workspace/bip-epigenomics-2.5.3-RLS/bip-epigenomics/'  # branch bip-epigenomics-2.5.3-RLS for sirius 1.6.1
    parameter_set_yaml = "/ghsfa/projects/pharma/customers/GuardantHealthIST/Infinity/MDC04_IST01_Infinity/MDC04_IST01_Infinity_01/data/rerun_crc_caller/bip-products/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml" # This is for Sirius 1.6.1

    # Directories for jobs and output 
    jobs_dir = os.path.join(workdir, "MB_module_CRC_jobs")
    output_base_dir = os.path.join(workdir, "MB_module_CRC_output")
    
    # Create directories if they don't exist
    os.makedirs(jobs_dir, exist_ok=True)
    os.makedirs(output_base_dir, exist_ok=True)
    
    for _, row in sample_meta_df.iterrows():
        run_sample_id = row["GHSampleID"]
        fc_dir = row["fc_dir"]
        sample_dir = f"{fc_dir}/{run_sample_id}/"
        
        # Define input files from sample_dir
        mbd_hyper_molecules = os.path.join(sample_dir, f"{run_sample_id}.mbd_hyper_molecules.tsv")
        methyl_qc_file = os.path.join(sample_dir, f"{run_sample_id}.methyl_qc.tsv")
        subpanel_qc_file = os.path.join(sample_dir, f"{run_sample_id}.subpanel_qc_stats.hdr.tsv")
        
        # Skip sample if the main input file is missing
        if not os.path.exists(mbd_hyper_molecules):
            continue
        
        # Create output directory for the sample (mimics out_path/sampleID)
        out_dir = os.path.join(output_base_dir, run_sample_id)
        os.makedirs(out_dir, exist_ok=True)
        
        # Copy bip_config.json from fc_dir to the sample's output directory
        bip_config_json_file = os.path.join(fc_dir, "bip_config.json")
        os.system(f"cp {bip_config_json_file} {out_dir}")
        
        # Create a job file that will run all three steps sequentially
        job_file = os.path.join(jobs_dir, f"MB_call_{run_sample_id}.job")
        
        # Refined SLURM header
        shebang_content = (
            "#!/bin/bash\n"
            f"#SBATCH --job-name=MB_call_{run_sample_id}\n"
            "#SBATCH --nodes=1\n"
            "#SBATCH --ntasks=1\n"
            "#SBATCH --cpus-per-task=16\n"
            "#SBATCH --mem=150g\n"
            "#SBATCH --partition=dev\n"
            f"#SBATCH --output={jobs_dir}/MB_call_{run_sample_id}.log\n"
            f"#SBATCH --error={jobs_dir}/MB_call_{run_sample_id}.err\n\n"
        )
        
        # 1. run msre_caller
        cmd_msre_caller = (
            f"{module_dir}/module_entry msre_caller "
            f"--in-hyper-molecules {mbd_hyper_molecules} "
            f"--sample-id {run_sample_id} "
            f"--output-dir {out_dir} "
            f"--parameter-set {parameter_set_yaml} "
            f"--threads 16 "
            f"--memory-log {out_dir}/{run_sample_id}-epigenomics_msre_caller.memory.log "
            f"--log-file {out_dir}/{run_sample_id}-epigenomics_msre_caller.log\n"
        )
        
        # 2. run epigenomics_tumor_fraction
        cmd_tumor_fraction = (
            f"{module_dir}/module_entry epigenomics_tumor_fraction "
            f"--in-hyper-molecules {mbd_hyper_molecules} "
            f"--in-methyl-qc-file {methyl_qc_file} "
            f"--in-subpanel-qc-file {subpanel_qc_file} "
            f"--sample-id {run_sample_id} "
            f"--output-dir {out_dir} "
            f"--parameter-set {parameter_set_yaml} "
            f"--threads 2 "
            f"--memory-log {out_dir}/{run_sample_id}-epigenomics_tumor_fraction.memory.log "
            f"--log-file {out_dir}/{run_sample_id}-epigenomics_tumor_fraction.log\n"
        )
        
        # 3. run collect_methyl_callers
        cmd_collect = (
            f"{module_dir}/module_entry collect_methyl_callers "
            f"--in-msre-crc-call {out_dir}/{run_sample_id}/{run_sample_id}.msre_crc_call.hdr.tsv "
            f"--in-msre-lung-call {out_dir}/{run_sample_id}/{run_sample_id}.msre_lung_call.hdr.tsv "
            f"--in-msre-breast-call {out_dir}/{run_sample_id}/{run_sample_id}.msre_breast_call.hdr.tsv "
            f"--in-msre-pancancer-call {out_dir}/{run_sample_id}/{run_sample_id}.msre_pancancer_call.hdr.tsv "
            f"--in-tf-estimate {out_dir}/{run_sample_id}/{run_sample_id}.epi_tf_model_predictions.tsv "
            f"--sample-id {run_sample_id} "
            f"--output {out_dir} "
            f"--parameter-set {parameter_set_yaml} "
            f"--threads 2 "
            f"--memory-log {out_dir}/{run_sample_id}-epigenomics_msre_summary.memory.log "
            f"--log-file {out_dir}/{run_sample_id}-epigenomics_msre_summary.log\n"
        )
        
        # Write all commands into the job file
        with open(job_file, "w") as jf:
            jf.write(shebang_content)
            jf.write(cmd_msre_caller)
            jf.write(cmd_tumor_fraction)
            jf.write(cmd_collect)
        
        # Submit the job file via sbatch
        os.system(f"sbatch {job_file}")


In [ ]:
# Instead of using epicaller.methylation.methyl_caller funciton, bip-epigenomics module (branch 2.5) is prefered.

def run_bip_epigenomics_module_wrapper_slurm_Sirius160(sample_meta_df, workdir):
    module_dir = '/home/mcai/workspace/bip-epigenomics/'  # branch bip-epigenomics-2.5.0-RC2 for sirius 1.6.0
    parameter_set_yaml = "/ghdevhome/home/hazhang/projects/Repo/bip-products_versions/bip-products-Sirius160/bip-products/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml" # This is for Sirius 1.6.0

    # Directories for jobs and output 
    jobs_dir = os.path.join(workdir, "MB_module_CRC_jobs")
    output_base_dir = os.path.join(workdir, "MB_module_CRC_output")
    
    # Create directories if they don't exist
    os.makedirs(jobs_dir, exist_ok=True)
    os.makedirs(output_base_dir, exist_ok=True)
    
    for _, row in sample_meta_df.iterrows():
        run_sample_id = row["GHSampleID"]
        fc_dir = row["fc_dir"]
        sample_dir = f"{fc_dir}/{run_sample_id}/"
        
        # Define input files from sample_dir
        mbd_hyper_molecules = os.path.join(sample_dir, f"{run_sample_id}.mbd_hyper_molecules.tsv")
        methyl_qc_file = os.path.join(sample_dir, f"{run_sample_id}.methyl_qc.tsv")
        subpanel_qc_file = os.path.join(sample_dir, f"{run_sample_id}.subpanel_qc_stats.hdr.tsv")
        
        # Skip sample if the main input file is missing
        if not os.path.exists(mbd_hyper_molecules):
            continue
        
        # Create output directory for the sample (mimics out_path/sampleID)
        out_dir = os.path.join(output_base_dir, run_sample_id)
        os.makedirs(out_dir, exist_ok=True)
        
        # Copy bip_config.json from fc_dir to the sample's output directory
        bip_config_json_file = os.path.join(fc_dir, "bip_config.json")
        os.system(f"cp {bip_config_json_file} {out_dir}")
        
        # Create a job file that will run all three steps sequentially
        job_file = os.path.join(jobs_dir, f"MB_call_{run_sample_id}.job")
        
        # Refined SLURM header
        shebang_content = (
            "#!/bin/bash\n"
            f"#SBATCH --job-name=MB_call_{run_sample_id}\n"
            "#SBATCH --nodes=1\n"
            "#SBATCH --ntasks=1\n"
            "#SBATCH --cpus-per-task=16\n"
            "#SBATCH --mem=150g\n"
            "#SBATCH --partition=dev\n"
            f"#SBATCH --output={jobs_dir}/MB_call_{run_sample_id}.log\n"
            f"#SBATCH --error={jobs_dir}/MB_call_{run_sample_id}.err\n\n"
        )
        
        # 1. run msre_caller
        cmd_msre_caller = (
            f"{module_dir}/module_entry msre_caller "
            f"--in-hyper-molecules {mbd_hyper_molecules} "
            f"--sample-id {run_sample_id} "
            f"--output-dir {out_dir} "
            f"--parameter-set {parameter_set_yaml} "
            f"--threads 16 "
            f"--memory-log {out_dir}/{run_sample_id}-epigenomics_msre_caller.memory.log "
            f"--log-file {out_dir}/{run_sample_id}-epigenomics_msre_caller.log\n"
        )
        
        # 2. run epigenomics_tumor_fraction
        cmd_tumor_fraction = (
            f"{module_dir}/module_entry epigenomics_tumor_fraction "
            f"--in-hyper-molecules {mbd_hyper_molecules} "
            f"--in-methyl-qc-file {methyl_qc_file} "
            f"--in-subpanel-qc-file {subpanel_qc_file} "
            f"--sample-id {run_sample_id} "
            f"--output-dir {out_dir} "
            f"--parameter-set {parameter_set_yaml} "
            f"--threads 2 "
            f"--memory-log {out_dir}/{run_sample_id}-epigenomics_tumor_fraction.memory.log "
            f"--log-file {out_dir}/{run_sample_id}-epigenomics_tumor_fraction.log\n"
        )
        
        # 3. run collect_methyl_callers
        cmd_collect = (
            f"{module_dir}/module_entry collect_methyl_callers "
            f"--in-msre-crc-call {out_dir}/{run_sample_id}/{run_sample_id}.msre_crc_call.hdr.tsv "
            f"--in-msre-lung-call {out_dir}/{run_sample_id}/{run_sample_id}.msre_lung_call.hdr.tsv "
            f"--in-msre-breast-call {out_dir}/{run_sample_id}/{run_sample_id}.msre_breast_call.hdr.tsv "
            f"--in-msre-pancancer-call {out_dir}/{run_sample_id}/{run_sample_id}.msre_pancancer_call.hdr.tsv "
            f"--in-tf-estimate {out_dir}/{run_sample_id}/{run_sample_id}.epi_tf_model_predictions.tsv "
            f"--sample-id {run_sample_id} "
            f"--output {out_dir} "
            f"--parameter-set {parameter_set_yaml} "
            f"--threads 2 "
            f"--memory-log {out_dir}/{run_sample_id}-epigenomics_msre_summary.memory.log "
            f"--log-file {out_dir}/{run_sample_id}-epigenomics_msre_summary.log\n"
        )
        
        # Write all commands into the job file
        with open(job_file, "w") as jf:
            jf.write(shebang_content)
            jf.write(cmd_msre_caller)
            jf.write(cmd_tumor_fraction)
            jf.write(cmd_collect)
        
        # Submit the job file via sbatch
        os.system(f"sbatch {job_file}")


In [ ]:
#run the function to generate all the MB S3 caller outputs
run_bip_epigenomics_module_wrapper_slurm_Sirius161(df_2, f"{work_dir}")

Submitted batch job 101496965
Submitted batch job 101496966
Submitted batch job 101496967
Submitted batch job 101496968
Submitted batch job 101496969
Submitted batch job 101496970
Submitted batch job 101496971


In [10]:
# Retrieve MB S3 CRC caller score and MB S3 Lung caller score based on the cancer type and stage
df_3 = df_2.copy()

df_3["MB_CRC_call_Sirius150"] = np.nan
df_3["MB_CRC_score_Sirius150"] = np.nan
df_3["pred_frac"] = np.nan

mb_caller_result_folder = f"{work_dir}/MB_module_CRC_output"
problematic_rows = []  # List to store indices of problematic rows

for index, crc_row in df_3.iterrows():
    run_sample_id = crc_row["GHSampleID"]
    msre_mr_file = f"{mb_caller_result_folder}/{run_sample_id}/{run_sample_id}/{run_sample_id}.msre_all_call.hdr.tsv"   
    try:
        msre_df = pd.read_table(msre_mr_file)
        msre_df_crc = msre_df[msre_df['model_name'] == 'CRC_LateStage']
        # print(msre_df_crc)

        if not msre_df_crc.empty:
            df_3.at[index, "pred_frac"] = msre_df_crc.iloc[0]["pred_frac"]
            df_3.at[index, "MB_CRC_score_Sirius150"] = msre_df_crc.iloc[0]["methyl_score"]
            df_3.at[index, "MB_CRC_call_Sirius150"] = "Detected" if msre_df_crc.iloc[0]["methyl_call"] == 1 else "Not Detected"
        else:
            # Add the index to the list if msre_df is empty
            problematic_rows.append(index) 
            
    except Exception as e:
        # Add the index to the list if there is any other exception
        problematic_rows.append(index)  
        print(f"Error processing file {msre_mr_file}: {str(e)}")

# Create a DataFrame with the indices of the problematic rows
df_problematic = df_3.loc[problematic_rows]

print("Total sample size of samples with NO MB output is " + str(df_problematic.shape[0]) + "\n") 

# Returning the DataFrame with problematic rows
df_3.head()


Total sample size of samples with NO MB output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid,fc_dir,MB_CRC_call_Sirius150,MB_CRC_score_Sirius150,pred_frac
2,XTX101-0102-001,NaN,A1243649,A124364901,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,21.39,NaN,NaN,12/18/24,12/17/24,1/2/25,241221_A01020_0945_AHWTCMDSXC,/ghsfa/ivd/flowcentral/241221_A01020_0945_AHWTCMDSXC/sirius,Not Detected,-21.906225,0.000000
3,XTX101-0102-001,NaN,A1247604,A124760401,146-005,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,26.64,NaN,NaN,12/21/24,12/19/24,1/2/25,241227_A02019_0258_BHWTKCDSXC,/ghsfa/ivd/flowcentral/241227_A02019_0258_BHWTKCDSXC/sirius,Detected,-20.906050,0.000270
5,XTX101-0102-001,NaN,A1263902,A126390201,141-006,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,11.10,NaN,NaN,1/11/25,1/9/25,2/4/25,250114_A01021_0837_AH27KNDSXF,/ghsfa/ivd/flowcentral/250114_A01021_0837_AH27KNDSXF/sirius,Detected,-20.515209,0.000103
4,XTX101-0102-001,NaN,A1276510,A127651001,140-008,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,33.94,NaN,NaN,1/23/25,1/22/25,2/11/25,250125_A01744_0377_AH27JKDSXF,/ghsfa/ivd/flowcentral/250125_A01744_0377_AH27JKDSXF/sirius,Not Detected,-21.774074,0.000000
0,XTX101-0102-001,NaN,A1277545,A127754501,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,19.86,NaN,NaN,1/24/25,1/23/25,2/11/25,250126_A01021_0847_BH272FDSXF,/ghsfa/ivd/flowcentral/250126_A01021_0847_BH272FDSXF.0e716317-4529-4481-89c6-10be60fc1006.20250128124242/sirius,Detected,-20.986001,0.000235


## Check the result

In [11]:
df_flipped_call = df_3[df_3['MB_CRC_call_Sirius150'] != df_3['Tumor_methylation_status']]
print("Total CRC samples with flipped call is " + str(df_flipped_call.shape[0]) + "\n")

# Save the data
# df_flipped_call.to_csv(f"{work_dir}/20250121_Pfizer_SOW51core_Batch3_18Samples_InfinityReport_flipped_calls.csv", index = False)
# df_flipped_call.to_csv(f"{work_dir}/20241220_GuardantHealthIST_IST01_Batch1_405Samples_InfinityReport_flipped_calls.csv", index = False)
# df_flipped_call.to_csv(f"{work_dir}/20250120_Pfizer_SOW46Amendment1TP3_Batch9_20Samples_G360Report_flipped_calls.csv", index = False)
# df_flipped_call.to_csv(f"{work_dir}/20250120_Pfizer_SOW46Amendment1TP5_Batch7_33Samples_InfinityReport_flipped_calls.csv", index = False)
df_flipped_call.to_csv(f"{work_dir}/XTX_monitoring_Sirius161_flipped_calls_by_cancer_stage_cutoff_03072025.csv", index = False)

df_flipped_call



Total CRC samples with flipped call is 5



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid,fc_dir,MB_CRC_call_Sirius150,MB_CRC_score_Sirius150,pred_frac
3,XTX101-0102-001,NaN,A1247604,A124760401,146-005,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,26.64,NaN,NaN,12/21/24,12/19/24,1/2/25,241227_A02019_0258_BHWTKCDSXC,/ghsfa/ivd/flowcentral/241227_A02019_0258_BHWTKCDSXC/sirius,Detected,-20.906050,0.000270
5,XTX101-0102-001,NaN,A1263902,A126390201,141-006,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,11.10,NaN,NaN,1/11/25,1/9/25,2/4/25,250114_A01021_0837_AH27KNDSXF,/ghsfa/ivd/flowcentral/250114_A01021_0837_AH27KNDSXF/sirius,Detected,-20.515209,0.000103
0,XTX101-0102-001,NaN,A1277545,A127754501,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,19.86,NaN,NaN,1/24/25,1/23/25,2/11/25,250126_A01021_0847_BH272FDSXF,/ghsfa/ivd/flowcentral/250126_A01021_0847_BH272FDSXF.0e716317-4529-4481-89c6-10be60fc1006.20250128124242/sirius,Detected,-20.986001,0.000235
1,XTX101-0102-001,NaN,A1285830,A128583001,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,27.23,NaN,NaN,1/31/25,1/30/25,2/19/25,250202_A00569_0940_AH3CM5DSXF,/ghsfa/ivd/flowcentral/250202_A00569_0940_AH3CM5DSXF/sirius,Detected,-21.213370,0.000045
6,XTX101-0102-001,NaN,A1218823,B00848429,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,13.21,3.5,NaN,11/22/24,11/21/24,12/17/24,241204_A01112_0657_BHTFJKDSXC,/ghsfa/ivd/flowcentral/241204_A01112_0657_BHTFJKDSXC/sirius,Detected,-21.248167,0.000099
